In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np

In [2]:
path = 'all_participants_result.xlsx'
df = pd.read_excel(path)
df

,ID,Essai,Temps Experience,Temps total Inactivite,Temps total Activite,Temps total de Manipulation,Nbr Total de Manipulation,Temps Total Consultation Instruction,Nb Total de Consultation,Nombre erreur,Proposition_Phase
0,ID10,1,96.099953,55.673687,40.426266,22.837461,23,17.58880,24,5,1
1,ID10,2,54.533314,31.585220,22.948090,22.248150,21,0.69994,2,1,2
2,ID10,3,41.166672,24.883051,16.283621,16.283621,15,0.00000,0,0,2
3,ID10,4,42.422218,22.697981,19.724239,19.724239,19,0.00000,0,0,3
4,ID10,5,33.722225,18.840277,14.881948,14.881948,17,0.00000,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...
384,ID9,3,53.544449,25.779648,27.764799,25.764799,14,2.00000,3,0,2
385,ID9,4,48.744450,23.301680,25.442772,24.987211,15,0.45556,1,0,2
386,ID9,5,43.344448,20.241940,23.102510,23.102510,14,0.00000,0,0,2
387,ID9,6,40.733341,21.255289,19.478050,19.478050,14,0.00000,0,0,3


In [34]:
#we wont use "propostion_phase"
features = ["Temps Experience", "Temps total Inactivite", "Temps total Activite",
            "Temps total de Manipulation", "Nbr Total de Manipulation",
            "Temps Total Consultation Instruction", "Nb Total de Consultation", "Nombre erreur"]

targets = ["Essai"]

In [44]:
features = ["Essai", "Temps Experience", "Temps total Inactivite", "Temps total Activite",
            "Temps total de Manipulation", "Nbr Total de Manipulation",
            "Temps Total Consultation Instruction"]

targets = ["Nb Total de Consultation", "Nombre erreur"]

In [86]:
X_train = df[features][df["ID"] != "ID55"]
y_train = df[targets][df["ID"] != "ID55"]

X_test = df[features][df["ID"] == "ID55"]
y_test = df[targets][df["ID"] == "ID55"]

model = LinearRegression()
model.fit(X_train, y_train)

predictions = model.predict(X_test)
predictions[predictions < 1] = 0
predictions_round = np.floor(predictions)
df_predictions = pd.DataFrame(predictions_round, columns=targets)

display(df_predictions.style.set_caption('Predictions'), y_test.style.set_caption('Real values'))

,Nb Total de Consultation,Nombre erreur
0,26.000000,4.000000
1,8.000000,8.000000
2,12.000000,1.000000
3,3.000000,0.000000
4,1.000000,0.000000
5,0.000000,0.000000


,Nb Total de Consultation,Nombre erreur
281,23,0
282,8,9
283,17,4
284,0,0
285,0,0
286,0,0


In [87]:
dummy_regressor = DummyRegressor(strategy='mean')
dummy_regressor.fit(X_train, y_train)
predictions_dummy = dummy_regressor.predict(X_test)

mse_dummy = mean_squared_error(y_test, predictions_dummy)
print(f"MSE Dummy Regressor : {mse_dummy}")
mse = mean_squared_error(y_test, predictions)
print(f"MSE with all attempts dataset: {mse}")
print(f"Linear regression is {100 - np.trunc((mse/mse_dummy)*100)}% better than a Dummy Regression")

MSE Dummy Regressor : 47.83048319914923
MSE with all attempts dataset: 6.354674424709907
Linear regression is 87.0% better than a Dummy Regression


In [88]:
X_train = df[features][df["ID"] != "ID60"]
y_train = df[targets][df["ID"] != "ID60"]

X_test = df[features][df["ID"] == "ID60"]
y_test = df[targets][df["ID"] == "ID60"]

model = LinearRegression()
model.fit(X_train, y_train)

predictions = model.predict(X_test)
predictions[predictions < 1] = 0
predictions_round = np.floor(predictions)
df_predictions = pd.DataFrame(predictions_round, columns=targets)

display(df_predictions.style.set_caption('Predictions'), y_test.style.set_caption('Real values'))

,Nb Total de Consultation,Nombre erreur
0,13.000000,1.000000
1,9.000000,1.000000
2,0.000000,0.000000
3,0.000000,0.000000
4,1.000000,0.000000
5,0.000000,0.000000
6,0.000000,0.000000
7,0.000000,0.000000


,Nb Total de Consultation,Nombre erreur
310,26,4
311,17,1
312,0,1
313,0,0
314,0,1
315,0,0
316,0,0
317,0,0


In [89]:
dummy_regressor = DummyRegressor(strategy='mean')
dummy_regressor.fit(X_train, y_train)
predictions_dummy = dummy_regressor.predict(X_test)

mse_dummy = mean_squared_error(y_test, predictions_dummy)
print(f"MSE Dummy Regressor : {mse_dummy}")
mse = mean_squared_error(y_test, predictions)
print(f"MSE with all attempts dataset: {mse}")
print(f"Linear regression is {100 - np.trunc((mse/mse_dummy)*100)}% better than a Dummy Regression")

MSE Dummy Regressor : 54.93810234842692
MSE with all attempts dataset: 14.314397851407769
Linear regression is 74.0% better than a Dummy Regression


## CROSS VALIDATION

In [94]:
participants = df['ID'].unique()

proportion_train = 0.8

train_size = int(len(participants) * proportion_train)

participants_train = np.random.choice(participants, size=train_size, replace=False)
participants_test = [id for id in participants if id not in participants_train]

print(f'Participants for training: {participants_train}')
print(f'Participants for testing: {participants_test}')
    
X_train = df[features][df["ID"].isin(participants_train)]
y_train = df[targets][df["ID"].isin(participants_train)]

X_test = []
y_test = []

mse_dummy = []
mse = []
model = LinearRegression()
model.fit(X_train, y_train)

for participant_test in participants_test:
    X_test = df[features][df["ID"] == participant_test]
    y_test = df[targets][df["ID"] == participant_test]

    predictions = model.predict(X_test)
    predictions[predictions < 1] = 0
    predictions_round = np.floor(predictions)
    df_predictions = pd.DataFrame(predictions_round, columns=targets)

    display(df_predictions.style.set_caption(f'Predictions for participant {participant_test}'), y_test.style.set_caption(f'Real values for participant {participant_test}'))

    dummy_regressor = DummyRegressor(strategy='mean')
    dummy_regressor.fit(X_train, y_train)
    predictions_dummy = dummy_regressor.predict(X_test)

    mse_dummy.append(mean_squared_error(y_test, predictions_dummy))
    mse.append(mean_squared_error(y_test, predictions))
    
avg_mse = np.mean(mse)
avg_mse_dummy = np.mean(mse_dummy)
print(f"MSE Dummy Regressor : {avg_mse_dummy}")
print(f"MSE with all attempts dataset: {avg_mse}")
print(f"Linear regression is {100 - np.trunc((avg_mse/avg_mse_dummy)*100)}% better than a Dummy Regression")

Participants for training: ['ID63' 'ID49' 'ID54' 'ID18' 'ID60' 'ID59' 'ID44' 'ID40' 'ID55' 'ID12'
 'ID53' 'ID51' 'ID41' 'ID24' 'ID69' 'ID33' 'ID70' 'ID38' 'ID28' 'ID36'
 'ID20' 'ID16' 'ID57' 'ID37' 'ID15' 'ID45' 'ID65' 'ID39' 'ID35' 'ID62'
 'ID17' 'ID19' 'ID50' 'ID71' 'ID21' 'ID23' 'ID64' 'ID34' 'ID67' 'ID14'
 'ID10' 'ID42' 'ID26' 'ID25' 'ID52' 'ID29' 'ID31' 'ID30']
Participants for testing: ['ID22', 'ID27', 'ID32', 'ID43', 'ID46', 'ID47', 'ID48', 'ID56', 'ID58', 'ID66', 'ID68', 'ID9']


,Nb Total de Consultation,Nombre erreur
0,73.000000,34.000000
1,14.000000,11.000000
2,1.000000,0.000000
3,1.000000,3.000000
4,3.000000,0.000000
5,1.000000,2.000000
6,0.000000,1.000000
7,0.000000,0.000000
8,0.000000,2.000000


,Nb Total de Consultation,Nombre erreur
66,86,44
67,13,6
68,0,0
69,0,1
70,8,0
71,0,1
72,0,0
73,0,0
74,0,0


,Nb Total de Consultation,Nombre erreur
0,38.000000,8.000000
1,10.000000,3.000000
2,6.000000,1.000000
3,1.000000,0.000000
4,1.000000,0.000000
5,0.000000,0.000000
6,0.000000,0.000000


,Nb Total de Consultation,Nombre erreur
102,25,8
103,7,0
104,5,0
105,1,0
106,0,0
107,0,0
108,0,0


...

,Nb Total de Consultation,Nombre erreur
0,132.000000,16.000000
1,26.000000,3.000000
2,10.000000,1.000000
3,3.000000,1.000000
4,0.000000,0.000000
5,0.000000,0.000000


,Nb Total de Consultation,Nombre erreur
359,116,4
360,35,0
361,16,0
362,0,0
363,0,0
364,0,0


,Nb Total de Consultation,Nombre erreur
0,58.000000,6.000000
1,4.000000,0.000000
2,1.000000,0.000000
3,0.000000,0.000000
4,0.000000,0.000000
5,0.000000,0.000000
6,0.000000,0.000000


,Nb Total de Consultation,Nombre erreur
382,36,7
383,7,0
384,3,0
385,1,0
386,0,0
387,0,0
388,0,0


MSE Dummy Regressor : 263.80607598123623
MSE with all attempts dataset: 20.700487233187236
Linear regression is 93.0% better than a Dummy Regression
